In [13]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
import MathOptInterface as MOI
import Ipopt 
import FiniteDiff
import ForwardDiff as FD
import Convex as cvx 
import ECOS
import MeshCat as mc
import Distributions
import Random

using LinearAlgebra
using Plots
using Random
using JLD2
using Test
using CSV
using DataFrames


  Activating environment at `c:\Users\AiPEX-WS4\Documents\AiPEX-Projects\warmstarting_NLPs\JL_presolves\Project.toml`


In [14]:
include(joinpath(@__DIR__, "utils","fmincon.jl"))
include(joinpath(@__DIR__, "utils","cartpole_animation.jl"))

animate_cartpole (generic function with 1 method)

In [61]:
# cartpole 
function dynamics(params::NamedTuple, x::Vector, u)
    # cartpole ODE, parametrized by params. 

    # cartpole physical parameters 
    mc, mp, l = params.mc, params.mp, params.l
    g = 9.81
    
    q = x[1:2]
    qd = x[3:4]

    s = sin(q[2])
    c = cos(q[2])


    H = [mc+mp mp*l*c; mp*l*c mp*l^2]
    @show size(H)
    C = [0 -mp*qd[2]*l*s; 0 0]
    G = [0, mp*g*l*s]
    B = [1, 0]

    qdd = -H\(C*qd + G - B*u[1])
    xdot = [qd;qdd]
    return xdot 

end

function hermite_simpson(params::NamedTuple, x1::Vector, x2::Vector, u, dt::Real)::Vector
    # TODO: input hermite simpson implicit integrator residual 
     x_mid = 0.5(x1 + x2) + (dt/8) * (dynamics(params, x1, u) - dynamics(params, x2, u))
     res = x1 + (dt/6) * (dynamics(params, x1, u) + 4*dynamics(params, x_mid, u) + dynamics(params, x2, u)) - x2
     return res
end

hermite_simpson (generic function with 1 method)

In [62]:
function create_idx(nx,nu,N)
    # This function creates some useful indexing tools for Z 
    # x_i = Z[idx.x[i]]
    # u_i = Z[idx.u[i]]
    
    # Feel free to use/not use anything here.
    
    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z 
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    
    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx 
    
    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x= x,u = u,c = c)
end

function cartpole_cost(params::NamedTuple, Z::Vector)::Real
    idx, N, xg = params.idx, params.N, params.xg
    Q, R, Qf = params.Q, params.R, params.Qf
    
    # TODO: input cartpole LQR cost 
    J = 0 

    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]
       
        J += 0.5*(xi-xg)'*Q*(xi-xg) + 0.5*ui'*R*ui
    end
    
    # dont forget terminal cost 
    xN = Z[idx.x[N]]
    J += 0.5*(xN-xg)'*Qf*(xN-xg)
    return J 
end

function cartpole_dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    
    # TODO: create dynamics constraints using hermite simpson 

    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), idx.nc)
    
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]] 
        xip1 = Z[idx.x[i+1]]
        
        # TODO: hermite simpson 
        c[idx.c[i]] = hermite_simpson(params, xi, xip1, ui, dt)
    end
    return c 
end

function cartpole_equality_constraint(params::NamedTuple, Z::Vector)::Vector
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 
    
    
    # TODO: return all of the equality constraints 

    
    return [Z[idx.x[1]] - xic; Z[idx.x[end]] - xg; cartpole_dynamics_constraints(params, Z)] 
end

function solve_cartpole_swingup(σ; verbose=true)
    
    # problem size 
    nx = 4 
    nu = 1 
    dt = 0.05
    tf = 2.0
    t_vec = 0:dt:tf 
    N = length(t_vec)
    
    # LQR cost 
    Q = 1*diagm(ones(nx))
    R = 0.1*diagm(ones(nu))
    Qf = 10*diagm(ones(nx))
    
    # indexing 
    idx = create_idx(nx,nu,N)
    
    # initial and goal states 
    # xic = [0, 0, 0, 0]
    xic = [σ[1], σ[2], 0, 0]
    xg = [0, pi, 0, 0]
    
    # load all useful things into params 
    params = (Q = Q, R = R, Qf = Qf, xic = xic, xg = xg, dt = dt, N = N, idx = idx,mc = 1.0, mp = 0.2, l = 0.5)
    
    # TODO: primal bounds 
    x_l = fill(-Inf, idx.nz)
    x_u = fill(Inf, idx.nz)
    
    for i = 1:(N-1)
        x_l[idx.u[i]] .= -10
        x_u[idx.u[i]] .= 10
    end

    
    # inequality constraint bounds (this is what we do when we have no inequality constraints)
    c_l = zeros(0)
    c_u = zeros(0)
    function inequality_constraint(params, Z)
        return zeros(eltype(Z), 0)
    end
    
    # initial guess 
    z0 = 0.01*randn(idx.nz)
    # z0 = pi*ones(idx.nz)
    # z0 = z0
    
    # choose diff type (try :auto, then use :finite if :auto doesn't work)
    diff_type = :auto 
#     diff_type = :finite
    
    # @show cartpole_equality_constraint(params, z0)
    # @show cartpole_dynamics_constraints(params, z0)
    # @show inequality_constraint(params, z0)
    # @show cartpole_cost(params, z0)
        
    Z, obj, solve_time_sec, term_status = fmincon(cartpole_cost,cartpole_equality_constraint,inequality_constraint,
                x_l,x_u,c_l,c_u,z0,params, diff_type;
                tol = 1e-6, c_tol = 1e-6, max_iters = 10_000, verbose = verbose)
    # term_status = 0




    # pull the X and U solutions out of Z 
    X = [Z[idx.x[i]] for i = 1:N]
    U = [Z[idx.u[i]] for i = 1:(N-1)]
    
    return X, U, obj, solve_time_sec, term_status, t_vec, params 
end


    


solve_cartpole_swingup (generic function with 1 method)

## Solve cartpole for a single parameter sample

In [63]:

σ = [0, pi, 0, 0]

X, U, obj, solve_time_sec, term_status, t_vec, params = solve_cartpole_swingup(σ; verbose=true)

# --------------testing------------------

Xm = hcat(X...)
Um = hcat(U...)

# --------------plotting-----------------
display(plot(t_vec, Xm', label = ["p" "θ" "ṗ" "θ̇"], xlabel = "time (s)", title = "State Trajectory"))
display(plot(t_vec[1:end-1],Um',label="",xlabel = "time (s)", ylabel = "u",title = "Controls"))




In [57]:
# display(animate_cartpole(X, 0.05))
animate_cartpole(X, 0.05)


┌ Info: Listening on: 127.0.0.1:8715, thread id: 1
└ @ HTTP.Servers C:\Users\AiPEX-WS4\.julia\packages\HTTP\sJD5V\src\Servers.jl:382
┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8715
└ @ MeshCat C:\Users\AiPEX-WS4\.julia\packages\MeshCat\0RCA3\src\visualizer.jl:64


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), UInt8[0x83, 0xa7, 0x6f, 0x70, 0x74, 0x69, 0x6f, 0x6e, 0x73, 0x82  …  0x5f, 0x61, 0x6e, 0x69, 0x6d, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("pole" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x73, 0x68, 0x63, 0x61, 0x74, 0x2f, 0x70, 0x6f, 0x6c, 0x65], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "b" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x6d, 0x65, 0x73, 0x68, 0x63, 0x61, 0x74, 0x2f, 0x62], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "cart" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x73, 0x68, 0x63, 0x61, 0x74, 0x2f, 0x63, 0x61, 0x72, 0x74], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), "a" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x2f, 0x6d, 0x65, 0x73, 0x68, 0x63, 0x61, 0x74, 0x2f, 0x61], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))), Set{HTTP.WebSockets.WebSocket}(), ip"127.0.0.1", 8715, HTTP.Servers.Server{HTTP.Servers.Listener{Nothing, Sockets.TCPServer}}(HTTP.Servers.Listener{Nothing, Sockets.TCPServer}(Sockets.InetAddr{Sockets.IPv4}(ip"127.0.0.1", 8715), "127.0.0.1", "8715", nothing, Sockets.TCPServer(Base.Libc.WindowsRawSocket(0x0000000000001378) active)), nothing, Set{HTTP.Connections.Connection}(), Task (runnable) @0x000000005d83a0e0, ReentrantLock(nothing, Base.GenericCondition{Base.Threads.SpinLock}(Base.InvasiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), 0))))

## Solve the DIRCOL NLP for a parameter set

In [19]:
# ## Define upper and lower bounds of the parameters for the Paramaetric Optimal Control Problem
# # using the xic of the cartpole
# using Random, Distributions, CSV, DataFrames, ProgressMeter
# Random.seed!(123)

# N = 50 # number of samples
# σ_lower = [-1.0, 0]
# σ_upper = [1.0, 2*pi]

# # Randomly sample the iid parameters uniformly from the given bounds
# d = Product(Uniform.(σ_lower, σ_upper))
# σ_samples = rand(d, N)
# σ_samples = eachcol(σ_samples)

# # Solve the NLP for the parameter sample set
# df = DataFrame(params = Vector{Vector{Float64}}(), X=Vector{Vector{Vector{Float64}}}(), U=Vector{Vector{Vector{Float64}}}(), obj = Float64[], solve_time_sec = Float64[], term_status = MOI.TerminationStatusCode[])

# p = Progress(N, 1)
# i = 1
# for σ in σ_samples
#     next!(p)
#     # z0 = 0.001*randn(idx.nz)
#     X, U, obj, solve_time_sec, term_status, t_vec, params = solve_cartpole_swingup(σ, verbose=false)

#     # if i % 10 == 0
#     #     println("Sample: ", i)
#     #     println("------------------")
#     #     println("σ: ", σ)
#     #     println("Objective Value: ", obj)
#     #     println("Termination Status Code: ", term_status)
#     #     println("Solve Time: ", solve_time_sec, "s")
#     #     println("")
#     # end

#     push!(df, [σ, X, U, obj, solve_time_sec, term_status])
#     i += 1
# end



Progress: 100%|█████████████████████████████████████████| Time: 0:01:34


In [20]:

# CSV.write("../data/presolves/cartpole_DIRCOL_1000_pi_wrap_fix.csv", df)

df

Row,params,X,U,obj,solve_time_sec,term_status
,Array…,Array…,Array…,Float64,Float64,Terminat…
1,"[0.536895, 5.90943]","[[0.536895, 3.37297, 1.29781e-13, -4.126e-13], [0.525056, 3.35546, -0.474051, -0.704672], [0.489399, 3.30165, -0.953515, -1.46086], [0.429539, 3.20769, -1.44276, -2.31824], [0.344945, 3.06746, -1.94226, -3.31627], [0.235296, 2.87354, -2.44192, -4.46246], [0.101232, 2.6196, -2.9119, -5.70227], [-0.0545727, 2.30364, -3.30267, -6.92384], [-0.205339, 1.95407, -2.72254, -7.14077], [-0.325965, 1.58137, -2.09028, -7.85149] … [-0.698828, 0.978451, 0.652881, 9.57426], [-0.646737, 1.41853, 1.41233, 8.11869], [-0.559374, 1.79922, 2.06735, 7.19345], [-0.441136, 2.1459, 2.65431, 6.75201], [-0.313006, 2.4587, 2.4556, 5.75849], [-0.201164, 2.71481, 2.00669, 4.48245], [-0.113088, 2.90742, 1.51208, 3.23326], [-0.0501441, 3.03969, 1.00532, 2.07466], [-0.01251, 3.11647, 0.50078, 1.00969], [-3.01424e-14, 3.14159, -6.90501e-14, 2.50241e-13]]","[[-10.0], [-10.0], [-10.0], [-10.0], [-10.0], [-10.0], [-10.0], [10.0], [10.0], [10.0] … [10.0], [10.0], [10.0], [10.0], [-6.35807], [-10.0], [-10.0], [-10.0], [-10.0], [-10.0]]",1299.07,1.745,LOCALLY_INFEASIBLE
2,"[0.347917, 2.48471]","[[0.347917, 2.48471, 0.0, 0.0], [0.335215, 2.44959, -0.506704, -1.40327], [0.306937, 2.35885, -0.622486, -2.23211], [0.28188, 2.23756, -0.379024, -2.63915], [0.273618, 2.09811, 0.0492363, -2.967], [0.286951, 1.93793, 0.486512, -3.47057], [0.322633, 1.74788, 0.945845, -4.16512], [0.382213, 1.51779, 1.44607, -5.07816], [0.468331, 1.23569, 2.01171, -6.25406], [0.577193, 0.893763, 2.3526, -7.43912] … [-0.229905, 2.08268, -0.86037, 3.07803], [-0.262148, 2.2273, -0.430171, 2.73624], [-0.272935, 2.35921, -0.000830958, 2.56917], [-0.26213, 2.48716, 0.434506, 2.57904], [-0.229292, 2.62025, 0.881373, 2.7776], [-0.175196, 2.76577, 1.28503, 3.07795], [-0.109182, 2.91517, 1.35642, 2.92416], [-0.0501441, 3.03969, 1.00532, 2.07466], [-0.01251, 3.11647, 0.50078, 1.00969], [0.0, 3.14159, 0.0, 6.9533e-31]]","[[-10.0], [-1.77467], [5.94236], [10.0], [10.0], [10.0], [10.0], [10.0], [2.97412], [-9.53868] … [10.0], [10.0], [10.0], [10.0], [10.0], [8.77879], [1.74058], [-6.85525], [-10.0], [-10.0]]",699.009,0.871,LOCALLY_SOLVED
3,"[-0.373512, 4.16295]","[[-0.373512, 3.35163, 1.33444e-13, -4.14319e-13], [-0.38542, 3.33336, -0.476804, -0.735123], [-0.421285, 3.27723, -0.959094, -1.52314], [-0.481493, 3.17933, -1.45103, -2.41408], [-0.566547, 3.03347, -1.95207, -3.44508], [-0.676661, 2.83243, -2.4498, -4.61658], [-0.810935, 2.5705, -2.91114, -5.86426], [-0.950872, 2.27088, -2.68094, -6.16458], [-1.0714, 1.95755, -2.13569, -6.43884], [-1.1637, 1.61995, -1.54587, -7.13774] … [-0.735627, 0.925806, 0.852823, 9.97188], [-0.672894, 1.38409, 1.63828, 8.45231], [-0.573651, 1.78047, 2.31562, 7.49246], [-0.445188, 2.14016, 2.81291, 6.96499], [-0.313006, 2.4587, 2.4556, 5.75849], [-0.201164, 2.71481, 2.00669, 4.48245], [-0.113088, 2.90742, 1.51208, 3.23326], [-0.0501441, 3.03969, 1.00532, 2.07466], [-0.01251, 3.11647, 0.50078, 1.00969], [-6.29864e-14, 3.14159, -7.00678e-15, 2.00705e-13]]","[[-10.0], [-10.0], [-10.0], [-10.0], [-10.0], [-10.0], [3.6008], [10.0], [10.0], [10.0] … [5.49854], [10.0], [10.0], [7.54446], [-10.0], [-10.0], [-10.0], [-10.0], [-10.0], [-10.0]]",1279.31,1.416,LOCALLY_INFEASIBLE
4,"[0.172044, 0.327562]","[[0.172044, 0.327562, 0.0, 6.18897e-29], [0.178888, 0.306778, 0.273761, -0.828773], [0.198008, 0.247916, 0.490764, -1.51705], [0.22705, 0.15792, 0.669845, -2.0674], [0.264389, 0.0439419, 0.821693, -2.46974], [0.30878, -0.0861919, 0.951041, -2.70847], [0.359099, -0.223878, 1.05856, -2.76974], [0.4142, -0.360019, 1.14281, -2.64844], [0.472852, -0.485606, 1.20169, -2.35242], [0.533721, -0.59234, 1.23265, -1.90064] … [-0.112594, 2.79031, 0.20602, 1.49437], [-0.100484, 2.86058, 0.279182, 1.32917], [-0.085486, 2.92275, 0.321492, 1.16921], [-0.0690205, 2.9771, 0.337849, 1.0151], [-0.052291, 3.02391, 0.331997, 0.866185], [-0.0363451, 3.06338, 0.306423, 0.719954], [-0.0221416, 3.09

## Load in the Warmstarts and Solve for the Refined Trajectories

In [21]:
# using CSV

# df = DataFrame(CSV.File("data/warmstart_cartpole.csv"))
# Z_warmstart_str = df.Z_warmstart[1]
# @show Z_warmstart_str
# Z_warmstart_str = replace(Z_warmstart_str, r"\n" => "")
# Z_warmstart = eval(Meta.parse(Z_warmstart_str))
# # # Function to convert the string representation to a 2D array
# # function convert_to_2d_array(str::String)
# #     # Remove the brackets and newline characters
# #     clean_str = replace(str, r"[\[\]\n]" => "")
# #     # Split the string into individual numbers
# #     num_strs = split(clean_str)
# #     # Convert the numbers to Float64
# #     nums = parse.(Float64, num_strs)
# #     # Reshape the flat array into a 2D array
# #     num_rows = count(x -> x == '\n', str) + 1
# #     num_cols = length(nums) ÷ num_rows
# #     return reshape(nums, num_cols, num_rows)'
# # end

# # X_warmstart = convert_to_2d_array(X_warmstart_str)
# # X_warmstart = X_warmstart'
# # X_warmstart = [X_warmstart[:, i] for i in 1:size(X_warmstart, 2)]
# # @show size(X_warmstart[1])

In [22]:
# # Assuming you have a DataFrame named df

# for row in eachrow(df)
#     Z_warmstart_str = row.Z_warmstart
#     Z_warmstart_str = replace(Z_warmstart_str, r"\n" => "")
#     Z_warmstart = vec(eval(Meta.parse(Z_warmstart_str)))
#     # @show typeof(Z_warmstart)
#     param_str = row.params
#     σ = vec(eval(Meta.parse(param_str)))
#     # @show typeof(vec(param_str))

#     X, U, obj, solve_time_sec, term_status, t_vec, params = solve_cartpole_swingup(σ, Z_warmstart; verbose=false)


# end